# Import Modules and Data

In [1]:
# Load modules
import sys
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [2]:
from notify_run import Notify
notify = Notify()
notify.register()

Endpoint: https://notify.run/zL31HO4uMjObX8Aj
To subscribe, open: https://notify.run/c/zL31HO4uMjObX8Aj
Or scan this QR code:

                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              
                                                              

In [3]:
# Load custom functions
sys.path.append('C:\\Users\\Dylan\\Desktop\\Data Science\\Projects\\DistractedDrivers\\functions')
from ddfuncs import cvmodeleval, samplecv, trainsampling

In [4]:
# Set memory limit on GPU to keep it from freezing up when fitting TensorFlow models later
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB * 2 of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0], \
                                                                [tf.config.experimental.\
                                                                 VirtualDeviceConfiguration\
                                                                 (memory_limit=1024 * 4)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [5]:
# Load processed training data
os.chdir('../data/processed')
df = pd.read_csv('driver_image_list_processed.csv')

In [6]:
df

,subject,classname,img,imgpath
0,p002,c0,img_44733.jpg,imgs/train/c0/img_44733.jpg
1,p002,c0,img_72999.jpg,imgs/train/c0/img_72999.jpg
2,p002,c0,img_25094.jpg,imgs/train/c0/img_25094.jpg
3,p002,c0,img_69092.jpg,imgs/train/c0/img_69092.jpg
4,p002,c0,img_92629.jpg,imgs/train/c0/img_92629.jpg
...,...,...,...,...
22419,p081,c9,img_56936.jpg,imgs/train/c9/img_56936.jpg
22420,p081,c9,img_46218.jpg,imgs/train/c9/img_46218.jpg
22421,p081,c9,img_25946.jpg,imgs/train/c9/img_25946.jpg
22422,p081,c9,img_67850.jpg,imgs/train/c9/img_67850.jpg


In [7]:
# Change to appropriate directory for data generation
os.chdir('../raw')

# Model 1 - 1 Conv, 1 MaxPool

In [ ]:
model1 = Sequential()
model1.add(Conv2D(10, kernel_size=3, activation='relu', input_shape=(256, 256, 3)))
model1.add(MaxPool2D(10))
model1.add(Flatten())
model1.add(Dense(10, activation='softmax'))
opt = Adam(learning_rate=0.0001)
model1.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model1.summary()

In [10]:
model1data = samplecv(model1, 
             df, 
             samples=80, 
             epochs=10, 
             steps_per_epoch=50, 
             validation_steps=None, 
             target_size=(256,256),
             random_state=42)

Resample iteration 1
CV iteration 1
Substep 1 of 26
Found 20000 validated image filenames belonging to 10 classes.
Found 800 validated image filenames belonging to 10 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 50 steps, validate for 25 steps
Epoch 1/10
50/50 [==============================] - 16s 314ms/step - loss: 78.6884 - accuracy: 0.1100 - val_loss: 67.9801 - val_accuracy: 0.0700
Epoch 2/10
50/50 [==============================] - 15s 293ms/step - loss: 38.0262 - accuracy: 0.1975 - val_loss: 69.9036 - val_accuracy: 0.0737
Epoch 3/10
50/50 [==============================] - 15s 296ms/step - loss: 26.6548 - accuracy: 0.2625 - val_loss: 63.5681 - val_accuracy: 0.0725
Epoch 4/10
50/50 [==============================] - 14s 286ms/step - loss: 20.1196 - accuracy: 0.3438 - val_loss: 68.8819 - val_accuracy: 0.0775
CV iteration 1
Substep 2 of 26
Found 20000 validated image filenames belonging to 10 classes.
Found 800 validated image filenames belonging to 10 classe

50/50 [==============================] - 15s 304ms/step - loss: 6.7932 - accuracy: 0.6094 - val_loss: 30.0435 - val_accuracy: 0.2850
CV iteration 1
Substep 13 of 26
Found 20000 validated image filenames belonging to 10 classes.
Found 800 validated image filenames belonging to 10 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 50 steps, validate for 25 steps
Epoch 1/10
50/50 [==============================] - 15s 299ms/step - loss: 82.8701 - accuracy: 0.1181 - val_loss: 70.1746 - val_accuracy: 0.1800
Epoch 2/10
50/50 [==============================] - 15s 296ms/step - loss: 31.2662 - accuracy: 0.2269 - val_loss: 48.2369 - val_accuracy: 0.1950
Epoch 3/10
50/50 [==============================] - 15s 294ms/step - loss: 23.1927 - accuracy: 0.2912 - val_loss: 43.0943 - val_accuracy: 0.1900
Epoch 4/10
50/50 [==============================] - 15s 294ms/step - loss: 16.8973 - accuracy: 0.3787 - val_loss: 40.0767 - val_accuracy: 0.2512
Epoch 5/10
50/50 [=====================

Found 20000 validated image filenames belonging to 10 classes.
Found 800 validated image filenames belonging to 10 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 50 steps, validate for 25 steps
Epoch 1/10
50/50 [==============================] - 14s 273ms/step - loss: 83.5133 - accuracy: 0.1300 - val_loss: 86.7403 - val_accuracy: 0.2000
Epoch 2/10
50/50 [==============================] - 14s 273ms/step - loss: 34.1064 - accuracy: 0.2156 - val_loss: 63.8836 - val_accuracy: 0.1375
Epoch 3/10
50/50 [==============================] - 14s 274ms/step - loss: 21.8628 - accuracy: 0.3200 - val_loss: 48.0223 - val_accuracy: 0.1787
Epoch 4/10
50/50 [==============================] - 14s 273ms/step - loss: 16.5071 - accuracy: 0.3781 - val_loss: 47.6916 - val_accuracy: 0.1525
CV iteration 1
Substep 25 of 26
Found 20000 validated image filenames belonging to 10 classes.
Found 800 validated image filenames belonging to 10 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...'

In [11]:
model1data

,iteration,validation_subject,val_acc1
0,1,p002,0.078
1,1,p012,0.309
2,1,p014,0.111
3,1,p015,0.169
4,1,p016,0.394
5,1,p021,0.148
6,1,p022,0.305
7,1,p024,0.442
8,1,p026,0.121
9,1,p035,0.345


In [12]:
model1data.to_csv('../metrics/model1metrics.csv', index=False)

# Model 2 - Add Dropout layer

In [16]:
model2 = Sequential()
model2.add(Conv2D(10, kernel_size=3, activation='relu', input_shape=(256, 256, 3)))
model2.add(Dropout(0.25))
model2.add(MaxPool2D(10))
model2.add(Flatten())
model2.add(Dense(10, activation='softmax'))
opt = Adam(learning_rate=0.0001)
model2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 10)      280       
_________________________________________________________________
dropout (Dropout)            (None, 254, 254, 10)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 25, 25, 10)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6250)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                62510     
Total params: 62,790
Trainable params: 62,790
Non-trainable params: 0
_________________________________________________________________


In [18]:
model2data = samplecv(model2, 
                      df, 
                      isampled=1, 
                      samples=80, 
                      n_iterations=1, 
                      epochs=10, 
                      steps_per_epoch=50, 
                      validation_steps=None, 
                      target_size=(256,256),
                      random_state=42)

Resample iteration 1
CV iteration 1
Substep 1 of 26
Found 20000 validated image filenames belonging to 10 classes.
Found 800 validated image filenames belonging to 10 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 50 steps, validate for 25 steps
Epoch 1/10
50/50 [==============================] - 15s 296ms/step - loss: 62.3729 - accuracy: 0.1488 - val_loss: 73.1160 - val_accuracy: 0.0700
Epoch 2/10
50/50 [==============================] - 14s 279ms/step - loss: 35.2610 - accuracy: 0.2362 - val_loss: 63.4384 - val_accuracy: 0.0763
Epoch 3/10
50/50 [==============================] - 14s 278ms/step - loss: 28.5700 - accuracy: 0.2537 - val_loss: 66.7993 - val_accuracy: 0.1000
Epoch 4/10
50/50 [==============================] - 14s 277ms/step - loss: 20.3414 - accuracy: 0.3562 - val_loss: 51.1579 - val_accuracy: 0.0988
CV iteration 1
Substep 2 of 26
Found 20000 validated image filenames belonging to 10 classes.
Found 800 validated image filenames belonging to 10 classe

50/50 [==============================] - 14s 274ms/step - loss: 33.8477 - accuracy: 0.2269 - val_loss: 20.0406 - val_accuracy: 0.3525
Epoch 3/10
50/50 [==============================] - 14s 274ms/step - loss: 23.2202 - accuracy: 0.3169 - val_loss: 19.3613 - val_accuracy: 0.3187
Epoch 4/10
50/50 [==============================] - 14s 274ms/step - loss: 17.1699 - accuracy: 0.3625 - val_loss: 18.6739 - val_accuracy: 0.3738
Epoch 5/10
50/50 [==============================] - 14s 274ms/step - loss: 14.7145 - accuracy: 0.4313 - val_loss: 13.7329 - val_accuracy: 0.5188
Epoch 6/10
50/50 [==============================] - 14s 275ms/step - loss: 11.6253 - accuracy: 0.5038 - val_loss: 14.3039 - val_accuracy: 0.4563
Epoch 7/10
50/50 [==============================] - 14s 274ms/step - loss: 9.8499 - accuracy: 0.5412 - val_loss: 13.1371 - val_accuracy: 0.4663
Epoch 8/10
50/50 [==============================] - 14s 274ms/step - loss: 8.7585 - accuracy: 0.5681 - val_loss: 10.6117 - val_accuracy: 0.542

Train for 50 steps, validate for 25 steps
Epoch 1/10
50/50 [==============================] - 14s 273ms/step - loss: 62.8465 - accuracy: 0.1650 - val_loss: 37.6315 - val_accuracy: 0.1975
Epoch 2/10
50/50 [==============================] - 14s 274ms/step - loss: 33.6460 - accuracy: 0.2350 - val_loss: 36.4091 - val_accuracy: 0.2075
Epoch 3/10
50/50 [==============================] - 14s 274ms/step - loss: 23.2577 - accuracy: 0.3000 - val_loss: 23.6727 - val_accuracy: 0.3288
Epoch 4/10
50/50 [==============================] - 14s 273ms/step - loss: 16.7427 - accuracy: 0.4019 - val_loss: 28.0936 - val_accuracy: 0.3225
Epoch 5/10
50/50 [==============================] - 14s 273ms/step - loss: 13.3068 - accuracy: 0.4775 - val_loss: 20.4603 - val_accuracy: 0.3200
Epoch 6/10
50/50 [==============================] - 14s 274ms/step - loss: 11.0663 - accuracy: 0.5263 - val_loss: 30.1336 - val_accuracy: 0.3212
CV iteration 1
Substep 21 of 26
Found 20000 validated image filenames belonging to 10 cl

In [19]:
model2data

,iteration,validation_subject,val_acc1
0,1,p002,0.100
1,1,p012,0.292
2,1,p014,0.122
3,1,p015,0.119
4,1,p016,0.226
5,1,p021,0.169
6,1,p022,0.202
7,1,p024,0.421
8,1,p026,0.156
9,1,p035,0.386


In [20]:
model2data.to_csv('../metrics/model2metrics.csv', index=False)

In [21]:
notify.send('model2 fitting complete')

Model overfitting

# Model 3 - Architecture Modeled off AlexNet

In [8]:
model3 = Sequential()
model3.add(Conv2D(99,
                 kernel_size=11,
                 strides=4,
                 padding='valid',
                 activation='relu',
                 input_shape=(227, 227, 3)))
model3.add(MaxPool2D(3,
                    strides=2,
                    padding='valid'))
model3.add(Conv2D(256,
                 kernel_size=5,
                 strides=1,
                 padding='same',
                 activation='relu'))
model3.add(MaxPool2D(3,
                    strides=2,
                    padding='valid'))
model3.add(Conv2D(384,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 activation='relu'))
model3.add(Conv2D(384,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 activation='relu'))
model3.add(Conv2D(256,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 activation='relu'))
model3.add(Flatten())
model3.add(Dense(100, activation='relu'))
model3.add(Dense(100, activation='relu'))
model3.add(Dense(10, activation='softmax'))
opt = Adam(learning_rate=0.00001)
model3.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 99)        36036     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 99)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       633856    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 384)       885120    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 384)       1327488   
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 256)       8

In [9]:
model3data = samplecv(model3, 
                      df, 
                      isampled=1, 
                      samples=80, 
                      n_iterations=1, 
                      epochs=50, 
                      steps_per_epoch=50, 
                      validation_steps=None, 
                      target_size=(227,227),
                      random_state=42)

Resample iteration 1
CV iteration 1
Substep 1 of 26
Found 20000 validated image filenames belonging to 10 classes.
Found 800 validated image filenames belonging to 10 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 50 steps, validate for 25 steps
Epoch 1/50
50/50 [==============================] - 16s 320ms/step - loss: 2.9296 - accuracy: 0.1469 - val_loss: 2.3337 - val_accuracy: 0.0538
Epoch 2/50
50/50 [==============================] - 14s 289ms/step - loss: 2.0473 - accuracy: 0.2975 - val_loss: 2.3590 - val_accuracy: 0.1762
Epoch 3/50
50/50 [==============================] - 14s 277ms/step - loss: 1.5500 - accuracy: 0.4812 - val_loss: 2.3224 - val_accuracy: 0.1250
Epoch 4/50
50/50 [==============================] - 14s 280ms/step - loss: 1.0471 - accuracy: 0.6450 - val_loss: 3.4854 - val_accuracy: 0.1525
Epoch 5/50
50/50 [==============================] - 17s 331ms/step - loss: 0.7829 - accuracy: 0.7412 - val_loss: 2.9962 - val_accuracy: 0.2313
Epoch 6/50
50/50 

Epoch 4/50
50/50 [==============================] - 13s 267ms/step - loss: 1.3717 - accuracy: 0.5081 - val_loss: 1.3664 - val_accuracy: 0.5500
Epoch 5/50
50/50 [==============================] - 13s 267ms/step - loss: 1.0346 - accuracy: 0.6381 - val_loss: 1.3668 - val_accuracy: 0.5788
Epoch 6/50
50/50 [==============================] - 13s 267ms/step - loss: 0.7155 - accuracy: 0.7594 - val_loss: 1.5118 - val_accuracy: 0.5425
Epoch 7/50
50/50 [==============================] - 13s 268ms/step - loss: 0.6815 - accuracy: 0.7819 - val_loss: 1.2749 - val_accuracy: 0.5412
Epoch 8/50
50/50 [==============================] - 13s 268ms/step - loss: 0.4566 - accuracy: 0.8556 - val_loss: 1.8050 - val_accuracy: 0.6075
Epoch 9/50
50/50 [==============================] - 13s 267ms/step - loss: 0.2916 - accuracy: 0.9225 - val_loss: 1.5661 - val_accuracy: 0.5725
Epoch 10/50
50/50 [==============================] - 13s 268ms/step - loss: 0.2175 - accuracy: 0.9294 - val_loss: 1.3771 - val_accuracy: 0.595

Epoch 11/50
50/50 [==============================] - 13s 268ms/step - loss: 0.2437 - accuracy: 0.9350 - val_loss: 1.6138 - val_accuracy: 0.5063
Epoch 12/50
50/50 [==============================] - 13s 269ms/step - loss: 0.1996 - accuracy: 0.9388 - val_loss: 1.2204 - val_accuracy: 0.5850
Epoch 13/50
50/50 [==============================] - 14s 273ms/step - loss: 0.2065 - accuracy: 0.9419 - val_loss: 2.1796 - val_accuracy: 0.4525
Epoch 14/50
50/50 [==============================] - 14s 271ms/step - loss: 0.2019 - accuracy: 0.9438 - val_loss: 0.9827 - val_accuracy: 0.6988
Epoch 15/50
50/50 [==============================] - 13s 268ms/step - loss: 0.1458 - accuracy: 0.9594 - val_loss: 1.3666 - val_accuracy: 0.5100
Epoch 16/50
50/50 [==============================] - 13s 267ms/step - loss: 0.1476 - accuracy: 0.9575 - val_loss: 1.1267 - val_accuracy: 0.6650
Epoch 17/50
50/50 [==============================] - 13s 268ms/step - loss: 0.1354 - accuracy: 0.9675 - val_loss: 0.8760 - val_accuracy:

50/50 [==============================] - 13s 268ms/step - loss: 0.3617 - accuracy: 0.8881 - val_loss: 0.7210 - val_accuracy: 0.8450
Epoch 9/50
50/50 [==============================] - 13s 268ms/step - loss: 0.3226 - accuracy: 0.9100 - val_loss: 0.6872 - val_accuracy: 0.8475
Epoch 10/50
50/50 [==============================] - 13s 268ms/step - loss: 0.2067 - accuracy: 0.9350 - val_loss: 0.8777 - val_accuracy: 0.7875
Epoch 11/50
50/50 [==============================] - 13s 268ms/step - loss: 0.1798 - accuracy: 0.9475 - val_loss: 0.7360 - val_accuracy: 0.8163
Epoch 12/50
50/50 [==============================] - 16s 329ms/step - loss: 0.1775 - accuracy: 0.9488 - val_loss: 0.6114 - val_accuracy: 0.8175
Epoch 13/50
50/50 [==============================] - 14s 287ms/step - loss: 0.1613 - accuracy: 0.9600 - val_loss: 1.4237 - val_accuracy: 0.6625
Epoch 14/50
50/50 [==============================] - 14s 288ms/step - loss: 0.1537 - accuracy: 0.9538 - val_loss: 0.9959 - val_accuracy: 0.7675
Epoch

Train for 50 steps, validate for 25 steps
Epoch 1/50
50/50 [==============================] - 14s 271ms/step - loss: 3.0452 - accuracy: 0.1037 - val_loss: 2.2884 - val_accuracy: 0.1000
Epoch 2/50
50/50 [==============================] - 14s 272ms/step - loss: 2.2828 - accuracy: 0.1431 - val_loss: 2.2592 - val_accuracy: 0.1863
Epoch 3/50
50/50 [==============================] - 14s 271ms/step - loss: 2.1808 - accuracy: 0.1925 - val_loss: 1.9012 - val_accuracy: 0.3825
Epoch 4/50
50/50 [==============================] - 14s 272ms/step - loss: 1.8889 - accuracy: 0.3256 - val_loss: 1.6373 - val_accuracy: 0.4875
Epoch 5/50
50/50 [==============================] - 14s 271ms/step - loss: 1.5615 - accuracy: 0.4431 - val_loss: 1.4261 - val_accuracy: 0.3837
Epoch 6/50
50/50 [==============================] - 14s 283ms/step - loss: 1.2523 - accuracy: 0.5537 - val_loss: 1.2630 - val_accuracy: 0.5600
Epoch 7/50
50/50 [==============================] - 14s 271ms/step - loss: 1.0309 - accuracy: 0.6419

Train for 50 steps, validate for 25 steps
Epoch 1/50
50/50 [==============================] - 13s 268ms/step - loss: 2.7871 - accuracy: 0.1163 - val_loss: 2.2939 - val_accuracy: 0.1000
Epoch 2/50
50/50 [==============================] - 13s 268ms/step - loss: 2.2574 - accuracy: 0.1644 - val_loss: 2.2207 - val_accuracy: 0.1637
Epoch 3/50
50/50 [==============================] - 14s 273ms/step - loss: 1.9239 - accuracy: 0.3338 - val_loss: 2.0579 - val_accuracy: 0.2663
Epoch 4/50
50/50 [==============================] - 14s 272ms/step - loss: 1.4346 - accuracy: 0.5075 - val_loss: 1.8414 - val_accuracy: 0.4087
Epoch 5/50
50/50 [==============================] - 16s 315ms/step - loss: 1.0817 - accuracy: 0.6475 - val_loss: 1.5501 - val_accuracy: 0.4238
Epoch 6/50
50/50 [==============================] - 14s 273ms/step - loss: 0.7703 - accuracy: 0.7544 - val_loss: 1.5072 - val_accuracy: 0.5050
Epoch 7/50
50/50 [==============================] - 14s 273ms/step - loss: 0.5631 - accuracy: 0.8075

In [10]:
model3data

,iteration,validation_subject,val_acc1
0,1,p002,0.231
1,1,p012,0.368
2,1,p014,0.469
3,1,p015,0.430
4,1,p016,0.861
5,1,p021,0.608
6,1,p022,0.619
7,1,p024,0.670
8,1,p026,0.668
9,1,p035,0.699


In [11]:
model3data.to_csv('../metrics/model3metrics.csv', index=False)

In [15]:
round(model3data.val_acc1.mean(), 3)

0.599

# Let's Try ResNet50

In [8]:
from tensorflow.keras.applications import DenseNet121

In [9]:
model4 = DenseNet121(include_top=True, weights=None, classes=10)
opt = Adam(learning_rate=0.01)
model4.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model4.summary()

Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________________________

In [12]:
model4data = samplecv(model4, 
                      df,
                      samples=80, 
                      batch_size=16,
                      epochs=50, 
                      steps_per_epoch=50, 
                      validation_steps=None, 
                      target_size=(299,299),
                      patience=25,
                      random_state=42)

Resample iteration 1
CV iteration 1
Substep 1 of 26
Found 20000 validated image filenames belonging to 10 classes.
Found 800 validated image filenames belonging to 10 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 50 steps, validate for 25 steps
Epoch 1/50
50/50 [==============================] - 40s 794ms/step - loss: 2.8602 - accuracy: 0.1000 - val_loss: 21135146844.1600 - val_accuracy: 0.1000
Epoch 2/50
50/50 [==============================] - 26s 524ms/step - loss: 2.3780 - accuracy: 0.1000 - val_loss: 3190.2396 - val_accuracy: 0.1000
Epoch 3/50
50/50 [==============================] - 33s 653ms/step - loss: 2.3868 - accuracy: 0.0988 - val_loss: 286.0759 - val_accuracy: 0.1088
Epoch 4/50
50/50 [==============================] - 26s 524ms/step - loss: 2.3199 - accuracy: 0.1013 - val_loss: 2.3200 - val_accuracy: 0.1000
Epoch 5/50
50/50 [==============================] - 26s 523ms/step - loss: 2.3099 - accuracy: 0.0938 - val_loss: 2.3050 - val_accuracy: 0.0925
Ep

KeyboardInterrupt: 

In [ ]:
notify.send('completed model 4 fitting')